In [1]:
import random
from collections import Counter
import stox
# from ib_insync import *
from stox.imports import *
from stox.ibkr import IB_Extractor, describe_contract, download_reports, IBKR_Loader, IBKR_Stats
from tqdm.notebook import tqdm
from omnibelt import tqdmd_notebook as tqdmd
from stox import misc, yahoo, ibkr
from collections import namedtuple
from omniply import tool, ToolKit, Context

In [2]:
symbol_table = stox.load_symbol_table()
len(symbol_table)

272

In [3]:
rows = [(k, v) for k, v in symbol_table.items() if v['currency'] == 'EUR']
len(rows)

239

In [4]:
date = '230829'
# date = 'last'

In [13]:
yfsym, ibrow = random.choice(rows)
gg = Context(IBKR_Loader(date=date, root=misc.ibkr_root()), IBKR_Stats())
gg.update(ibrow)
gg['ibsym'] = ibrow['symbol']
gg['ibid'] = ibrow['conId']
gg['yfsym'] = yfsym
print(yfsym)

KeyboardInterrupt: 

In [6]:
gg

Context(conId, currency, exchange, primaryExchange, symbol, ibsym, ibid, yfsym, {ckpt_date}, {ckpt_path}, {snapshot}, {ownership}, {finances}, {statements}, {recommendations}, {company_name}, {exchange_name}, {country}, {isin}, {clprice}, {price}, {market_cap}, {high_52w}, {low_52w}, {sector}, {employees}, {employees_date}, {business_summary}, {brief}, {brief_date}, {country_code}, {country_name}, {city}, {website}, {industry_trbc}, {industry_trbc_code}, {industry}, {industry_naics_code}, {industry_sic}, {industry_sic_code})

In [7]:
gg['company_name']

'AEGON N.V.'

{'@Major': '1',
 '@Minor': '0',
 '@Revision': '1',
 'CoIDs': {'CoID': [{'@Type': 'RepNo', '#text': 'A475E'},
   {'@Type': 'CompanyName', '#text': 'AEGON N.V.'},
   {'@Type': 'OrganizationPermID', '#text': '4295884931'}]},
 'Issues': {'Issue': [{'@ID': '1',
    '@Type': 'C',
    '@Desc': 'Common Stock',
    '@Order': '1',
    'IssueID': [{'@Type': 'Name', '#text': 'Ordinary Shares'},
     {'@Type': 'Ticker', '#text': 'AGN'},
     {'@Type': 'ISIN', '#text': 'NL0000303709'},
     {'@Type': 'RIC', '#text': 'AEGN.AS'},
     {'@Type': 'DisplayRIC', '#text': 'AEGN.AS'},
     {'@Type': 'InstrumentPI', '#text': '271052'},
     {'@Type': 'QuotePI', '#text': '717150'},
     {'@Type': 'InstrumentPermID', '#text': '8590934513'},
     {'@Type': 'QuotePermID', '#text': '55838224545'}],
    'Exchange': {'@Code': 'AMS',
     '@Country': 'NLD',
     '#text': 'Euronext Amsterdam Stock Exchange'},
    'GlobalListingType': 'OSR'},
   {'@ID': '4',
    '@Type': 'C',
    '@Desc': 'Common Stock',
    '@Order':

In [ ]:
Money = namedtuple('Money', 'amount currency')

class IBKR_Loader(TestCraftyKitBase):
	def __init__(self, date='last', root=None):
		super().__init__()
		if root is None:
			root = misc.ibkr_root()
		self.root = root
		self.date = date
	
	@tool('ckpt_date')
	def get_ckpt_date(self, conId):
		path = misc.get_date_path(self.root / str(conId), self.date)
		return path.name

	@tool('ckpt_path')
	def get_ckpt_path(self, conId, ckpt_date):
		return misc.get_date_path(self.root / str(conId), ckpt_date)

	def _load_xml(self, path):
		with open(path, 'r') as f:
			return xmltodict.parse(f.read())

	@tool('snapshot')
	def load_snapshot(self, ckpt_path):
		return self._load_xml(ckpt_path / 'snapshot.xml')
		
	@tool('ownership')
	def load_ownership(self, ckpt_path):
		return self._load_xml(ckpt_path / 'ownership.xml')
		
	@tool('finances')
	def load_finances(self, ckpt_path):
		return self._load_xml(ckpt_path / 'finances.xml')
	
	@tool('statements')
	def load_statements(self, ckpt_path):
		return self._load_xml(ckpt_path / 'statements.xml')
		
	@tool('recommendations')
	def load_recommendations(self, ckpt_path):
		return self._load_xml(ckpt_path / 'recommendations.xml')
		

class IBKR_Stats(TestCraftyKitBase):
	@tool('company_name')
	def get_company_name_from_snapshot(self, snapshot):
		val = snapshot['ColIDs']['CoID'][1]
		assert val['@type'] == 'CompanyName', f'Expected CompanyName in {val}'
		return val['#text']
	@tool('company_name')
	def get_company_name_from_rec(self, recommendations):
		return recommendations['Company']['CoName']['Name']['#text']
	
	@tool('exchange')
	def get_exchange_from_snapshot(self, snapshot):
		val = snapshot['Issues']['Issue']['Exchange']
		return val['@Code']
	@tool('exchange')
	def get_exchange_from_rec(self, recommendations):
		return recommendations['Company']['SecurityInfo']['Security']['Exchange']['@code']
	
	@tool('exchange_name')
	def get_exchange_name(self, snapshot):
		val = snapshot['Issues']['Issue']['Exchange']
		return val['#text']
	
	@tool('country')
	def get_country_code_from_snapshot(self, snapshot):
		val = snapshot['Issues']['Issue']['Exchange']
		return val['@Country']
	@tool('country')
	def get_country_code_from_rec(self, recommendations):
		loc = recommendations['Company']['SecurityInfo']['Security']['SecIds']['SecId'][0]
		assert loc['@type'] == 'ISIN', f'Expected ISIN in {loc}'
		return loc['#text']
	
	@tool('isin')
	def get_isin_from_snapshot(self, snapshot):
		val = snapshot['Issues']['Issue']['IssueID'][2]
		assert val['@Type'] == 'ISIN', f'Expected ISIN in {val}'
		return val['#text']
	@tool('isin')
	def get_isin_from_rec(self, recommendations):
		return recommendations['Company']['SecurityInfo']['Security']['ISIN']['#text']
	@tool('clprice')
	def get_clprice(self, recommendations):
		return recommendations['Company']['SecurityInfo']['Security']['@clprice']
	@tool('price')
	def get_price(self, recommendations):
		entry = recommendations['Company']['SecurityInfo']['MarketData']['MarketDataItem'][0]
		assert entry['@unit'] == 'U' and entry['@type'] == 'CLPRICE', f'Expected CLPRICE in {entry}'
		return Money(float(entry['#text']), entry['@currCode'])
	@tool('market_cap')
	def get_market_cap(self, recommendations):
		entry = recommendations['Company']['SecurityInfo']['MarketData']['MarketDataItem'][1]
		assert entry['@unit'] == 'M' and entry['@type'] == 'MARKETCAP', f'Expected MARKETCAP in {entry}'
		return Money(float(entry['#text']) * 1e6, entry['@currCode'])
	@tool('high_52w')
	def get_high_52w(self, recommendations):
		entry = recommendations['Company']['SecurityInfo']['MarketData']['MarketDataItem'][2]
		assert entry['@unit'] == 'U' and entry['@type'] == '52WKHIGH', f'Expected 52WKHIGH in {entry}'
		return Money(float(entry['#text']), entry['@currCode'])
	@tool('low_52w')
	def get_low_52w(self, recommendations):
		entry = recommendations['Company']['SecurityInfo']['MarketData']['MarketDataItem'][3]
		assert entry['@unit'] == 'U' and entry['@type'] == '52WKLOW', f'Expected 52WKLOW in {entry}'
		return Money(float(entry['#text']), entry['@currCode'])
	@tool('sector')
	def get_sector(self, recommendations):
		return recommendations['Company']['CompanyInfo']['Sector']['#text']
	
	@tool('employees')
	def get_employees(self, snapshot):
		val = snapshot['CoGeneralInfo']['Employees']
		return int(val['#text'])
	@tool('employees_date')
	def get_employees_date(self, snapshot):
		val = snapshot['CoGeneralInfo']['Employees']
		return val['@LastUpdated']
	
	@tool('business_summary')
	def get_company_summary(self, snapshot):
		val = snapshot['TextInfo']['Text'][0]
		assert val['@Type'] == 'Business Summary', f'Expected Business Summary in {val}'
		return val['#text']
	
	@tool('brief')
	def get_brief(self, snapshot):
		val = snapshot['TextInfo']['Text'][1]
		assert val['@Type'] == 'Financial Summary', f'Expected Financial Summary in {val}'
		return val['#text']
	@tool('brief_date')
	def get_brief_date(self, snapshot):
		val = snapshot['TextInfo']['Text'][1]
		assert val['@Type'] == 'Financial Summary', f'Expected Financial Summary in {val}'
		return val['@LastModified']
	
	@tool('country_code')
	def get_country(self, snapshot):
		val = snapshot['contactInfo']['country']
		return val['@code']
	@tool('country_name')
	def get_country_name(self, snapshot):
		val = snapshot['contactInfo']['country']
		return val['#text']
	@tool('city')
	def get_city(self, snapshot):
		val = snapshot['contactInfo']['city']
		return val['#text']
	# skipped address and phone and contact person info and email
	@tool('website')
	def get_website(self, snapshot):
		val = snapshot['webLinks']['webSite']
		return val['#text']
	
	@tool('industry_trbc')
	def get_industry_trbc(self, snapshot):
		full = snapshot['peerInfo']['IndustryInfo']['Industry']
		vals = [row for row in full if row['@type'] == 'TRBC']
		assert len(vals) == 1, f'Expected 1 TRBC in {vals}'
		return vals[0]['#text']
	@tool('industry_trbc_code')
	def get_industry_trbc_code(self, snapshot):
		full = snapshot['peerInfo']['IndustryInfo']['Industry']
		vals = [row for row in full if row['@type'] == 'TRBC']
		assert len(vals) == 1, f'Expected 1 TRBC in {vals}'
		return vals[0]['@code']
	
	@tool('industry')
	@tool('industry_naics')
	def get_industry_naics(self, snapshot):
		full = snapshot['peerInfo']['IndustryInfo']['Industry']
		vals = [row for row in full if row['@type'] == 'NAICS']
		assert len(vals), f'Expected NAICS in {vals}'
		return [v['#text'] for v in vals]
	@tool('industry_naics_code')
	def get_industry_naics_code(self, snapshot):
		full = snapshot['peerInfo']['IndustryInfo']['Industry']
		vals = [row for row in full if row['@type'] == 'NAICS']
		assert len(vals), f'Expected NAICS in {vals}'
		return [v['@code'] for v in vals]
	
	@tool('industry_sic')
	def get_industry_sic(self, snapshot):
		full = snapshot['peerInfo']['IndustryInfo']['Industry']
		vals = [row for row in full if row['@type'] == 'SIC']
		assert len(vals), f'Expected SIC in {vals}'
		return [v['#text'] for v in vals]
	@tool('industry_sic_code')
	def get_industry_sic_code(self, snapshot):
		full = snapshot['peerInfo']['IndustryInfo']['Industry']
		vals = [row for row in full if row['@type'] == 'SIC']
		assert len(vals), f'Expected SIC in {vals}'
		return [v['@code'] for v in vals]